# 数据导入&预处理

In [1]:
# 导入相关的库
import pandas as pd # 数据表
import numpy as np  # 数组
import re           # 正则表达式
import jieba        # 中文分词
import matplotlib.pyplot as plt   # 画图
from gensim import corpora, models # 用于构建LDA模型
import pyLDAvis  # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis # 适配gensim的pyLDAvis工具

In [2]:
# 导入微博文本数据
df = pd.read_excel(r'D:\text analysis\weibo_pure.xlsx')

In [3]:
# 显示DataFrame的前5行，初步了解数据结构和内容
df.head

<bound method NDFrame.head of                                                      内容
0     天呐！我被渣了？！恋爱无数次 竟然又遇渣男为了你们不被骗海后黎女士教你们三步识渣男！快快收藏...
1     难怪恐婚的越来越多，结婚的越来越少！看了这结婚协议，真是惊呆了！⭕️年轻人对待爱情都这么冷漠...
2     奋不顾身的投入一段感情但却被伤透了心或许这就是年轻人恐婚的源头吧！视频来源｜乔七月L蟹老板爱...
3     爸爸：“你不结婚，他不结婚，那人类不灭绝了吗？”00后女儿：“那恐龙也生孩子，不也灭绝了吗。...
4     我大概是一个很自私的人，我觉得结婚和生小孩会耽误我爱自己，也有可能是觉得自身能力支撑不住对自...
...                                                 ...
1041                               南京的小可爱们，你觉得有哪些呢？2南京​
1042  在一回首间，才忽然发现，原来，我一生的种种努力，不过只为了要使周遭的人都对我满意而已。为了要...
1043                       单身万岁！无拘无束！想干啥干啥！不用担心别人的想法！ ​
1044                                 胆小（既然是年轻人就应该多尝试） ​
1045                        一来觉得靠别人不如靠自己，再有就是觉得可能不长久吧 ​

[1046 rows x 1 columns]>

# 语料预处理

## 剔除符号和数字

In [4]:
# 定义文本清洗函数
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+','',text)  # 移除非中文字符的正则表达式操作
    return nonums

# 测试清洗函数
remove_nums(df['内容'][0])[:100]

'天呐我被渣了恋爱无数次竟然又遇渣男为了你们不被骗海后黎女士教你们三步识渣男快快收藏全是干货恋爱渣男黎拉的坏品味的微博视频抽奖详情'

## 分词 

In [5]:
# 文本清洗与分词函数
def clean_text(text):
    #剔除符号与数字，只保留汉字
    processed = re.sub('[^\u4e00-\u9fa5]+','',text)
    #分词
    words = jieba.lcut(processed)
    #剔除停用词
    stopwords = ['恐婚', '结婚', '婚姻', '没有', '年轻人', '一个', 
    '我们', '他们', '其实', '很多', '不会', '不能',
    '知道', '看到', '需要', '感觉', '如果', '那个', '已经', '还有', '怎么', '遇到',
    '应该', '想要', '认为', '或者', '甚至', '一直', '不要', '所有', '永远', '不敢',
    '只是', '只有', '虽然', '面对', '特别', '一种', '可是', '各种', '还要', '以为',
    '好像', '成为', '真正', '这些', '不过', '带来', '必须', '无论', '每个', '足够',
    '出现', '反而', '变成', '多少', '容易', '愿意', '一切', '不了', '不够', '东西',
    '于是', '怎样', '一些', '不到', '不愿', '原来', '只会', '告诉', '导致', '放弃',
    '确定', '结束', '结果', '而且', '讨论', '那种', '麻烦', '一点', '不仅', '之间',
    '大家', '如何', '总是', '我会', '曾经', '状态', '程度', '经历', '解决', '除了',
    '问题', '事情', '视频', '两个', '对方', '希望', '今天', '每天', '突然', '一样',
    '不如', '有人', '发现', '超话', '而已', '那些', '重要', '想法', '技巧', '准备',
    '微博', '觉得', '原因', '真的', '为什么', '因为', '什么', '就是', '不是', '但是',
    '可能', '这个', '那么', '现在', '可以', '不想', '害怕', '还是', '所以',
    '这样', '当代', '为了', '老公', '男人', '最后', '主要', '然后', '究竟', '产生',
    '一定', '这么', '这种', '大概', '身边', '一起', '喜欢', '时候','我怕', '开始'
    #单字停用词
    '人', '不', '怕', '被', '说', '还', '想', '也', '吧', '到', '多', '太', '没', '去',
    '看', '家', '给', '小', '把', '会', '后', '岁', '要', '的', '是', '了', '和', '就', 
    '都', '而', '及', '与', '着', '也', '还', '或', '被', '但', '你', '我', '他', '她', 
    '它', '我们', '他们','她们', '自己', '对', '啊', '呢', '吗', '很', '更', '又', '得', 
    '让', '才', '从', '中', '上', '下', '后', '前', '着', '再', '啊', '哪', '呢','里', 
    '这', '那', '个', '一', '不', '好', '做', '过', '能', '吧', '才', '跟', '哪', '什么', 
    '会', '人', '说', '么', '想', '在', '有', '无', '为','以', '到', '其', '来', '着', 
    '个', '着', '将', '等', '被', '并', '而', '此', '即', '每', '则', '于', '等', '之', 
    '及','的', '了', '和', '是', '就', '都', '而', '及', '与', '这', '那', '在', '有',
    '却', '我', '你', '他', '她', '它', '谁', '听', '地', '嘛', '可', '另', '当', '吃'
]
    words = [w for w in words if w not in stopwords]
    #return words
    return ' '.join(words)

# 测试函数
print(df['内容'][0][:20])  
print(clean_text(df['内容'][0])[:20])  

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SHANGS~1\AppData\Local\Temp\jieba.cache


天呐！我被渣了？！恋爱无数次 竟然又遇渣


Loading model cost 0.946 seconds.
Prefix dict has been built successfully.


天呐 渣 恋爱 无数次 竟然 遇渣 男 


In [6]:
# 应用到整个数据集
df['内容分词'] = df['内容'].astype(str).apply(clean_text)
# 转化为词语列表
df['内容分词'] = df['内容分词'].apply(lambda x: x.split())
# 显示结果
df['内容分词'] 

0       [天呐, 渣, 恋爱, 无数次, 竟然, 遇渣, 男, 你们, 骗海, 后黎, 女士, 教,...
1       [难怪, 越来越, 越来越少, 协议, 真是, 惊呆, 对待, 爱情, 冷漠, 数字化, 按...
2       [奋不顾身, 投入, 一段, 感情, 伤, 透了心, 或许, 源头, 来源, 乔, 七月, ...
3                 [爸爸, 人类, 灭绝, 女儿, 恐龙, 生, 孩子, 灭绝, 爸爸, 情感]
4                [自私, 生小孩, 耽误, 爱, 自身, 能力, 支撑, 不住, 小孩, 期待]
                              ...                        
1041                                 [南京, 小可爱, 们, 哪些, 南京]
1042    [回首, 间, 忽然, 一生, 种种, 努力, 只, 使, 周遭, 满意, 博得, 他人, ...
1043               [单身, 万岁, 无拘无束, 干, 啥, 干, 啥, 不用, 担心, 别人]
1044                                         [胆小, 既然, 尝试]
1045                               [一来, 靠, 别人, 靠, 再有, 长久]
Name: 内容分词, Length: 1046, dtype: object

# LDA

In [7]:
# 正式根据分词结果创建字典
dictionary = corpora.Dictionary(df['内容分词']) 
#根据分词结果创建语料库
corpus = [dictionary.doc2bow(text) for text in df['内容分词']] 

#dictionary
#corpus
# 显示词典中前10个词的映射关系（word → id）
print(list(dictionary.token2id.items())[:10])
# 显示前1条语料的词袋表示
print(corpus[0][:10])

[('三步', 0), ('你们', 1), ('全是', 2), ('后黎', 3), ('品味', 4), ('坏', 5), ('天呐', 6), ('女士', 7), ('干货', 8), ('快快', 9)]
[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]


In [8]:
# 训练LDA模型
# 使用gensim库中的LdaModel来训练LDA主题模型
# corpus：语料库（词袋格式）
# num_topics=5：设定模型提取5个主题
# id2word=dictionary：语料对应的词典（将词语id映射为原词）
# passes=15：设置训练的轮数（越高越稳定，但耗时更久）
lda_model = models.LdaModel(corpus, num_topics = 5, id2word=dictionary, passes = 15)

In [9]:
# 查看主题
topics = lda_model.print_topics(num_words=5)  # 显示每个主题中权重最高的 5 个关键词

# 遍历打印每个主题及其对应的关键词
for topic in topics:
    print(topic)
# 每个主题由一组关键词+权重组合构成，用于理解该主题的语义

(0, '0.008*"孩子" + 0.005*"妈妈" + 0.004*"生活" + 0.004*"恐惧" + 0.004*"男"')
(1, '0.010*"孩子" + 0.006*"单身" + 0.005*"恐育" + 0.005*"越来越" + 0.005*"爱"')
(2, '0.007*"女性" + 0.005*"生活" + 0.005*"钱" + 0.004*"幸福" + 0.004*"社会"')
(3, '0.009*"孩子" + 0.007*"爱情" + 0.006*"家庭" + 0.005*"吵架" + 0.005*"生活"')
(4, '0.009*"生活" + 0.008*"离婚" + 0.007*"孩子" + 0.005*"家暴" + 0.005*"爱"')


## 查看主题分布

In [10]:
# 查看某条微博属于哪个主题
#df['内容分词'].iloc[0]   #选取第0行/第1条微博数据
print(df['内容分词'].iloc[0][:5])  # 前5个分词

['天呐', '渣', '恋爱', '无数次', '竟然']


In [11]:
# 返回主题分布列表(主题编号, 概率)
for index, score in sorted(lda_model[corpus[0]],key=lambda tup: -1*tup[1]):
    print("\nSore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))  # 每个主题展示前10个词


Sore: 0.9712401628494263	 
Topic: 0.010*"孩子" + 0.006*"单身" + 0.005*"恐育" + 0.005*"越来越" + 0.005*"爱" + 0.004*"谈恋爱" + 0.004*"恋爱" + 0.004*"以后" + 0.004*"离婚" + 0.004*"爱情"


In [12]:
# 推理函数：给定文档返回主题分布
def infer_topics(lda_model,document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

# 遍历查看前10条微博的主题分布
documents = df["内容分词"].values.tolist()
#for i, doc in enumerate(documents[:10]):
    #print(doc)
    #doc_topics = infer_topics(lda_model, doc)
    #print(f"Document{i+1}:")
    #print(doc_topics)
    #print()

for i, doc in enumerate(documents[:10]):
    print(f"Document{i+1} 前5个词: {doc[:5]}")
    doc_topics = infer_topics(lda_model, doc)
    # 只展示概率最高的主题
    top_topic = sorted(doc_topics, key=lambda x: -x[1])[0]
    print(f"  => 最相关主题: Topic {top_topic[0]}, 概率: {top_topic[1]:.2f}")
    print()

Document1 前5个词: ['天呐', '渣', '恋爱', '无数次', '竟然']
  => 最相关主题: Topic 1, 概率: 0.97

Document2 前5个词: ['难怪', '越来越', '越来越少', '协议', '真是']
  => 最相关主题: Topic 1, 概率: 0.96

Document3 前5个词: ['奋不顾身', '投入', '一段', '感情', '伤']
  => 最相关主题: Topic 3, 概率: 0.95

Document4 前5个词: ['爸爸', '人类', '灭绝', '女儿', '恐龙']
  => 最相关主题: Topic 1, 概率: 0.58

Document5 前5个词: ['自私', '生小孩', '耽误', '爱', '自身']
  => 最相关主题: Topic 3, 概率: 0.93

Document6 前5个词: ['谈恋爱', '本质', '风险', '厌恶', '轻人']
  => 最相关主题: Topic 0, 概率: 0.98

Document7 前5个词: ['爱情', '主义', '失去', '妥协', '能力']
  => 最相关主题: Topic 3, 概率: 0.98

Document8 前5个词: ['单身', '越来越', '单身', '青年', '数量']
  => 最相关主题: Topic 1, 概率: 0.98

Document9 前5个词: ['很难', '难', '经济', '压力', '难']
  => 最相关主题: Topic 1, 概率: 0.64

Document10 前5个词: ['可怕', '一天', '动手术', '之际', '另一半']
  => 最相关主题: Topic 2, 概率: 0.98



# 可视化

### 每个主题之间的相似性（通过圆圈距离表示）
### 每个主题的关键词分布（右侧关键词表）
### 各主题在语料中的权重大小（圆圈大小）
### 可视化界面λ滑块控制关键词的排序依据：λ = 1.0：高频词；λ = 0.0：根据“提升度”排序（主题独有词）；λ = 0.6（默认）：综合

In [13]:
import pyLDAvis.gensim_models as gensimvis
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
# 显示可视化界面
pyLDAvis.display(lda_vis)

In [14]:
# 导出可视化结果到html
pyLDAvis.save_html(lda_vis, r'D:\text analysis\lda_visualization.html')